In [1]:
# Libraries
import pickle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from Datasets.ClsDatasets import WineDataset
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, UniformIntegerHyperparameter, \
    CategoricalHyperparameter, Constant
from HyperparametersOptimization.hyperparemeters_optimization import TunerSMAC
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from AutomaticModelSelection.automatic_model_selection import Arm, EfficientCASHRB
from copy import deepcopy

In [2]:
# Data
data = WineDataset()
X = data.input
Y = data.target
print(X.shape, Y.shape)

(178, 13) (178,)


In [11]:
# Hyperparameter(s)
hp_dict_adaboost = dict(
    n_estimators=UniformIntegerHyperparameter(name="n_estimators", lower=50, upper=500, default_value=50),
    learning_rate=UniformFloatHyperparameter(name="learning_rate", lower=0.01, upper=3, default_value=0.1),
    algorithm=CategoricalHyperparameter(name="algorithm", choices=["SAMME.R", "SAMME"], default_value="SAMME.R")
)

hp_dict_rf = dict(
    n_estimators=Constant("n_estimators", 100),
    criterion=CategoricalHyperparameter(name="criterion", choices=["gini", "entropy"], default_value="gini"),
    max_depth=UniformIntegerHyperparameter(name="max_depth", lower=1, upper=200, default_value=30),
)

hp_dict_knn = dict(
    n_neighbors=UniformIntegerHyperparameter(name="n_neighbors", lower=1, upper=100, default_value=1),
    weights=CategoricalHyperparameter(name="weights", choices=["uniform", "distance"], default_value="uniform"),
    p=CategoricalHyperparameter(name="p", choices=[1, 2], default_value=2)
)

hp_dict_smbo = dict(
    n_neighbors=UniformIntegerHyperparameter(name="n_neighbors", lower=1, upper=100, default_value=1),
    weights=CategoricalHyperparameter(name="weights", choices=["uniform", "distance"], default_value="uniform"),
    p=CategoricalHyperparameter(name="p", choices=[1, 2], default_value=2),
    #n_estimators_rf=Constant("n_estimators", 100),
    criterion=CategoricalHyperparameter(name="criterion", choices=["gini", "entropy"], default_value="gini"),
    max_depth=UniformIntegerHyperparameter(name="max_depth", lower=1, upper=200, default_value=30),
    n_estimators=UniformIntegerHyperparameter(name="n_estimators", lower=50, upper=500, default_value=50),
    learning_rate=UniformFloatHyperparameter(name="learning_rate", lower=0.01, upper=3, default_value=0.1),
    algorithm=CategoricalHyperparameter(name="algorithm", choices=["SAMME.R", "SAMME"], default_value="SAMME.R"),
    root=CategoricalHyperparameter(name="root", choices=[0, 1, 2], default_value=0)
)

In [15]:
# Objective(s)
def objective_adaboost(config):
    model = AdaBoostClassifier(n_estimators=config["n_estimators"], learning_rate=config["learning_rate"],
                               algorithm=config["algorithm"])
    scores = cross_val_score(model, X, Y, cv=10)
    print(scores.mean(), scores.std())
    return 1 - scores.mean()

def objective_rf(config):
    model = RandomForestClassifier(max_depth=config["max_depth"], criterion=config["criterion"], n_estimators=config["n_estimators"])
    scores = cross_val_score(model, X, Y, cv=10)
    print(scores.mean(), scores.std())
    return 1 - scores.mean()

def objective_knn(config):
    model = KNeighborsClassifier(n_neighbors=config["n_neighbors"], weights=config["weights"], p=config["p"])
    scores = cross_val_score(model, X, Y, cv=10)
    print(scores.mean(), scores.std())
    return 1 - scores.mean()

def objective_smbo(config):
    models = [
        AdaBoostClassifier(n_estimators=config["n_estimators"], learning_rate=config["learning_rate"],
                               algorithm=config["algorithm"]),
        RandomForestClassifier(max_depth=config["max_depth"], criterion=config["criterion"], n_estimators=config["n_estimators"]),
        KNeighborsClassifier(n_neighbors=config["n_neighbors"], weights=config["weights"], p=config["p"])
    ]
    model = models[config["root"]]
    print("ALGO: ", config["root"])
    scores = cross_val_score(model, X, Y, cv=10)
    print(scores.mean(), scores.std())
    return 1 - scores.mean()

In [16]:
# Tuner(s)
base_dir = "../experiments/Test_EfficientCASHRB/"
tuner_args = dict(
    hp_dict=deepcopy(hp_dict_adaboost),
    objective_foo=objective_adaboost,
    trials=1,
    log_path=base_dir + "test_ada",
    n_jobs=1,
    seed=2023
)
tuner_adaboost = TunerSMAC(**tuner_args)

tuner_args["hp_dict"] = deepcopy(hp_dict_rf)
tuner_args["objective_foo"] = objective_rf
tuner_args["log_path"] = base_dir + "test_rf"
tuner_rf = TunerSMAC(**tuner_args)

tuner_args["hp_dict"] = deepcopy(hp_dict_knn)
tuner_args["objective_foo"] = objective_knn
tuner_args["log_path"] = base_dir + "test_knn"
tuner_knn = TunerSMAC(**tuner_args)

tuner_args["hp_dict"] = deepcopy(hp_dict_smbo)
tuner_args["objective_foo"] = objective_smbo
tuner_args["log_path"] = base_dir + "test_smbo"
tuner_smbo = TunerSMAC(**tuner_args)

In [ ]:
# Arm(s)
arm_adaboost = Arm(model=AdaBoostClassifier, tuner=tuner_adaboost)
arm_rf = Arm(model=RandomForestClassifier, tuner=tuner_rf)
arm_knn = Arm(model=KNeighborsClassifier, tuner=tuner_knn)

# Dictionary of Arm(s)
arms_dict = dict(
    adaboost=arm_adaboost,
    random_forest=arm_rf,
    knn=arm_knn
)

In [ ]:
# Automatic Block
auto_model_generation = EfficientCASHRB(
    budget=40,
    train_data_input=X,
    train_data_output=Y,
    arm_dictionary=arms_dict,
    trials_per_step=2
)

In [ ]:
model = auto_model_generation.learn()
filename = base_dir + 'best_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [17]:
tuner_smbo.tune(30)

ALGO:  2
0.742483660130719 0.11563087185612614
ALGO:  0
0.596078431372549 0.09982256033395177
ALGO:  1
0.9777777777777779 0.027216552697590882
ALGO:  0
0.8833333333333334 0.08766518798921943
ALGO:  2
0.7869281045751634 0.08799046158194204
ALGO:  1
0.9833333333333334 0.02545875386086579
ALGO:  1
0.9777777777777779 0.027216552697590882
ALGO:  2
0.7202614379084966 0.09067317222666926
ALGO:  0
0.619281045751634 0.1014362450170186
ALGO:  1
0.9777777777777779 0.027216552697590882
ALGO:  1
0.9722222222222221 0.02777777777777779
ALGO:  1
0.9777777777777779 0.027216552697590882
ALGO:  1
0.9777777777777779 0.027216552697590882
ALGO:  1
0.9833333333333332 0.02545875386086579
ALGO:  2
0.7869281045751633 0.07674939398817016
ALGO:  1
0.9833333333333332 0.02545875386086579
ALGO:  2
0.7039215686274509 0.08704351275600779
ALGO:  1
0.9722222222222221 0.02777777777777779
ALGO:  1
0.9888888888888889 0.022222222222222233
ALGO:  1
0.9833333333333334 0.02545875386086579
ALGO:  1
0.9722222222222221 0.02777777

Configuration(values={
  'algorithm': 'SAMME',
  'criterion': 'entropy',
  'learning_rate': 0.08307331584931957,
  'max_depth': 150,
  'n_estimators': 118,
  'n_neighbors': 44,
  'p': 2,
  'root': 1,
  'weights': 'uniform',
})